In [48]:
# Instalando o pycaret

!pip3 install pycaret[full]

In [49]:
# Importando o pandas

import pandas as pd

In [50]:
# Lendo o dataset(https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset)

df = pd.read_csv("heart.csv")

In [51]:
# Primeiros cinco dados do dataset

df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [52]:
# Importando as bibliotecas do sklearn para fazer um modelo na mão

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [53]:
# Definindo os dados de teste para o modelo

X = df.drop(columns="output")
y = df.output

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=3)
print(X.shape)
print(y.shape)


(303, 13)
(303,)


In [54]:
# Pre-processando os dados

colunas_numericas = ['trtbps', 'chol', 'thalachh', 'oldpeak']

scaler = StandardScaler()

df[colunas_numericas] = scaler.fit_transform(df[colunas_numericas])

In [55]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, confusion_matrix, accuracy_score

In [56]:
# Analisando o modelo

regressao_logistica = LogisticRegression()
regressao_logistica.fit(X_train,y_train)
y_pred_RL = regressao_logistica.predict(X_test)

# Model Evaluation
print("Treino: ", regressao_logistica.score(X_train,y_train))
print("Teste: ", regressao_logistica.score(X_test,y_test))
print("MAE", mean_absolute_error(y_test,y_pred_RL))
print("MSE" , mean_squared_error(y_test,y_pred_RL))

print("Acurácia" , accuracy_score(y_test,y_pred_RL))

Treino:  0.8458149779735683
Teste:  0.868421052631579
MAE 0.13157894736842105
MSE 0.13157894736842105
Acurácia 0.868421052631579


In [57]:
# Importando o Pycaret para fazer o modelo com ele também

from pycaret.regression import *

In [58]:
# Visualizando os modelos com o Pycaret

s = setup(
    data=df,
    target = "output",
    session_id = 64
)

,Description,Value
0,Session id,64
1,Target,output
2,Target type,Regression
3,Original data shape,"(303, 14)"
4,Transformed data shape,"(303, 14)"
5,Transformed train set shape,"(212, 14)"
6,Transformed test set shape,"(91, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [59]:
# Comparando os modelos do Pycaret

melhor_modelo = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2658,0.1239,0.3489,0.4774,0.2451,0.2355,0.1100
rf,Random Forest Regressor,0.2571,0.1305,0.3582,0.4453,0.2493,0.2380,0.2750
br,Bayesian Ridge,0.2990,0.1324,0.3610,0.4385,0.2557,0.2671,0.1290
ridge,Ridge Regression,0.3000,0.1342,0.3633,0.4307,0.2562,0.2693,0.0890
lr,Linear Regression,0.3004,0.1345,0.3637,0.4292,0.2563,0.2699,1.2960
lar,Least Angle Regression,0.2997,0.1358,0.3655,0.4238,0.2568,0.2677,0.1150
catboost,CatBoost Regressor,0.2694,0.1356,0.3644,0.4203,0.2532,0.2511,0.1400
huber,Huber Regressor,0.3021,0.1405,0.3718,0.4053,0.2630,0.2578,0.2050
et,Extra Trees Regressor,0.2443,0.1409,0.3716,0.3950,0.2586,0.2162,0.2260
gbr,Gradient Boosting Regressor,0.2752,0.1412,0.3726,0.3944,0.2587,0.2565,0.1800


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [60]:
# Visualizando o melhor modelo

melhor_modelo

LGBMRegressor(n_jobs=-1, random_state=64)

In [61]:
# Vendo os dados do melhor modelo

evaluate_model(melhor_modelo)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [62]:
# Importando o Pickle para

import pickle

In [63]:
# Salvando o modelo criado fora do pycaret

pickle.dump(regressao_logistica, open('model.pkl', 'wb'))

In [64]:
# Salvando o modelo do pycaret

save_model(melhor_modelo, 'model_pycaret')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['age', 'sex', 'cp', 'trtbps',
                                              'chol', 'fbs', 'restecg',
                                              'thalachh', 'exng', 'oldpeak',
                                              'slp', 'caa', 'thall'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', LGBMRegressor(n_jobs=-1, random_state=64))]),
 'model_pycaret.pkl')

In [65]:
# Criando API com o modelo do Pycaret

create_api(melhor_modelo, 'minha_api')

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python minha_api.py
